## Optimization code for PV microgrid:

In [1]:
### Initialization:

using JuMP, Gurobi, MATLAB


In [2]:
my_start =  1
my_end   = 96

# my_start = 96
# my_end   = 96

96

In [3]:
# for my_int = my_start:my_end

my_int = my_start

while my_int <= my_end

    #----------------------------------------------------------------------------------------------------
    ### Solver Definition:
    
    m = Model(solver=GurobiSolver(Presolve=1,TimeLimit=900))
        
    #----------------------------------------------------------------------------------------------------
    ### Loading Optimization Parameters:

    mat"generate_data($my_int)"
    
    vars = read_matfile("../../Data/Generated Data/5 - Optimization/scenarios/scen_"*dec(my_int)*".mat")

    interval  = Int64(jvalue(vars["interval"]))  # Current Interval (in day, for which we are optimizing)

    N_EMS   = Int64(jvalue(vars["N_EMS"]))   # num of remaining EMS   intervals (opt. executions)
    N_intra = Int64(jvalue(vars["N_intra"])) # num of remaining Intra intervals (opt. resolution)
    N_dies  = Int64(jvalue(vars["N_dies"]))  # num of diesel generators
    N_scen  = Int64(jvalue(vars["N_scen"]))  # num of different scenations
    N_firm  = Int64(jvalue(vars["N_firm"]))  # num of FIRM Margin Constraint Intervals

    # Upper and Lower bounds:
    P_bat_min = jvalue(vars["P_bat_min"])
    P_bat_max = jvalue(vars["P_bat_max"])
    eta_bat  = jvalue(vars["eta_bat"])
    SOC_min  = jvalue(vars["SOC_min"])
    SOC_max  = jvalue(vars["SOC_max"])
    SOC_init = jvalue(vars["SOC_init"])
    Cap_bat  = jvalue(vars["Cap_bat"])
    P_dies_max = jvalue(vars["P_dies_max"])
    P_dies_min = jvalue(vars["P_dies_min"])
    P_PV_inst = jvalue(vars["P_PV_inst"])

    delta_t = jvalue(vars["delta_t"]) # 1 min

    # f_min = -100000
    f_min   = jvalue(vars["f_min"])
    
    theta_i = jvalue(vars["theta_i"])
    theta_d = jvalue(vars["theta_d"])
    theta_b = jvalue(vars["theta_b"])
    theta_p = jvalue(vars["theta_p"])
    
    U_min_def = jvalue(vars["U_min_def"]) # - my_correct*0.025
    
    psi_min_i = jvalue(vars["psi_min_i"])
    psi_min_d = jvalue(vars["psi_min_d"])
    psi_min_p = jvalue(vars["psi_min_p"])
    psi_min_b = jvalue(vars["psi_min_b"])
    psi_min_c = jvalue(vars["psi_min_c"])    
    
    psi_loss_i = jvalue(vars["psi_loss_i"])
    psi_loss_d = jvalue(vars["psi_loss_d"])
    psi_loss_p = jvalue(vars["psi_loss_p"])
    psi_loss_b = jvalue(vars["psi_loss_b"])
    psi_loss_c = jvalue(vars["psi_loss_c"])
    
    marge_dies = jvalue(vars["marge_dies"])
    
    L_PV = jvalue(vars["L_PV"]) # generated PV scenarios
    L_C  = jvalue(vars["L_C"]) # generated load scenarios
    L_C2 = reshape(mean(L_C,3),N_EMS,N_intra)

    # Defining iterators:
    ts  = 1:N_EMS
    ps  = 1:N_intra
    ds  = 1:N_dies
    ss  = 1:N_scen
    m1s = 1:N_firm
    m2s = (N_firm+1):N_EMS
    
    #----------------------------------------------------------------------------------------------------
    ### Variable definitions:

    @variables m begin

        # Battery variables:
        P_bat_min <= P_bat_set[t=ts,p=ps]  <= P_bat_max # Battery setpoint for time t,p (decision var.)
        0 <= P_bat_cha[t=ts,p=ps] <= P_bat_max
        0 <= P_bat_dis[t=ts,p=ps] <= P_bat_max
        X_bat[t=ts,p=ps], Bin                           # Operating mode indicator (decision var.)
                                                        # -> P_bat_set = 2*(X_bat-0.5)*P_bat_abs
        SOC_min <= SOC_bat[t=ts,p=ps] <= SOC_max        # Obtained SOC for time t,p (solution var.)

        # PV Plant:
        0 <= P_PV_set[t=ts,p=ps]  <= P_PV_inst          # PV setpoint variable (decision var.)
        0 <= P_PV[t=ts,p=ps,s=ss] <= L_PV[t,p,s]        # Actual PV generation (solution var.)

        # Diesel Generators:
        ON_dies[t=ts,d=ds], Bin                         # Dies. connectin setpoint (decision var.)
        0 <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max  # Actual dies. generation  (solution var.)
        # P_dies_min <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max # Actual dies. generation  (solution var.)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Variable Initialization (from previous optimization)
    
    if my_int>1

        vars = read_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(my_int-1)*".mat")

        P_bat_set_pre = jvalue(vars["P_bat_set"])[2:N_EMS,ps]
        SOC_bat_pre   = jvalue(vars["SOC_bat"])[2:N_EMS,ps]
        P_PV_set_pre  = jvalue(vars["P_PV_set"])[2:N_EMS,ps]
        P_PV_pre      = jvalue(vars["P_PV"])[2:N_EMS,ps,ss]
        ON_dies_pre   = jvalue(vars["ON_dies"])[2:N_EMS,ds]
        P_dies_pre    = jvalue(vars["P_dies"])[2:N_EMS,ps,ss,ds]
        P_bat_cha_pre = jvalue(vars["P_bat_cha"])[2:N_EMS,ps]
        P_bat_dis_pre = jvalue(vars["P_bat_dis"])[2:N_EMS,ps]
        X_bat_pre     = jvalue(vars["X_bat"])[2:N_EMS,ps]

        setvalue(P_bat_set[1:(N_EMS-1),ps],P_bat_set_pre)
        setvalue(SOC_bat[1:(N_EMS-1),ps],SOC_bat_pre)
        setvalue(P_PV_set[1:(N_EMS-1),ps],P_PV_set_pre)
        setvalue(P_PV[1:(N_EMS-1),ps,ss],P_PV_pre)
        setvalue(ON_dies[1:(N_EMS-1),ds],ON_dies_pre)
        setvalue(P_dies[1:(N_EMS-1),ps,ss,ds],P_dies_pre)
        setvalue(P_bat_cha[1:(N_EMS-1),ps],P_bat_cha_pre)
        setvalue(P_bat_dis[1:(N_EMS-1),ps],P_bat_dis_pre)
        setvalue(X_bat[1:(N_EMS-1),ps],X_bat_pre)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Constraints introduction:
    
    @constraints m begin

        # Constraints:
        #
        # A1 - Diesel power limitation
        # A2 - Diesel power limitation
        # B1 - Power balance equation
        # C1 - PV setpoint limitation
        # D1 - Bat. setpoint equation
        # D2 - Charging    mode const.
        # D3 - Discharging mode const.
        # E1 - FIRM    Diesel reserve margin (separated over scenarios)
        # E2 - RELAXED Diesel reserve margin (mean over scenarios)
        # F1 - 
        # F2 - SOC limitations
        # F3 - 
        # G1 - Minimum frequency limitation
        # X1 - Minimum voltage relaxed limitation
        # X2 - Minimum voltage firm    limitation

        A1[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] <= ON_dies[t,d]*(P_dies_max)
        A2[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] >= ON_dies[t,d]*(P_dies_min)

        # B1[t=ts,p=ps,s=ss], (P_PV[t,p,s] + sum(P_dies[t,p,s,d] for d=ds) + P_bat_set[t,p] - L_C[t,p,s]) == 0
        B1[t=ts,p=ps,s=ss], (P_PV[t,p,s]*(1-psi_loss_p) + sum((P_dies[t,p,s,d] - psi_loss_d*1e3*ON_dies[t,d]) for d=ds) + P_bat_set[t,p]*(1-psi_loss_b) - L_C[t,p,s]*(1+psi_loss_c) - psi_loss_i*1e3) == 0
                
        C1[t=ts,p=ps,s=ss],  P_PV[t,p,s] <= P_PV_set[t,p]

        D1[t=ts,p=ps], P_bat_set[t,p] == P_bat_cha[t,p] - P_bat_dis[t,p]
        D2[t=ts,p=ps], P_bat_cha[t,p] <= P_bat_max *    X_bat[t,p]
        D3[t=ts,p=ps], P_bat_dis[t,p] <= P_bat_max * (1-X_bat[t,p])

        # E1[t=ts,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E1[t=m1s,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E2[t=m2s,p=ps],  sum(P_dies[t,p,s,d] for d=ds,s=ss)/N_scen <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies

        F1[t=1,p=1],               SOC_bat[t,p] == SOC_init             - P_bat_set[t,p]*delta_t/Cap_bat
        F2[t=2:N_EMS,p=1],         SOC_bat[t,p] == SOC_bat[t-1,N_intra] - P_bat_set[t,p]*delta_t/Cap_bat
        F3[t=1:N_EMS,p=2:N_intra], SOC_bat[t,p] == SOC_bat[t,p-1]       - P_bat_set[t,p]*delta_t/Cap_bat

        G1[t=ts,p=ps], f_min <= theta_i + theta_d*sum(ON_dies[t,d] for d=ds) + theta_b*P_bat_set[t,p] + theta_p*P_PV_set[t,p]
        
        # X1[t=ts ,p=ps,s=ss], psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*P_PV[t,p,s] + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*L_C[t,p,s] >= U_min_def
        # X1[t=m1s,p=ps,s=ss], psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*P_PV[t,p,s]                      + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*L_C[t,p,s]                      >= U_min_def
        # X2[t=m2s,p=ps],      psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*sum(P_PV[t,p,s] for s=ss)/N_scen + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*sum(L_C[t,p,s] for s=ss)/N_scen >= U_min_def
        
    end

    #----------------------------------------------------------------------------------------------------
    ### Objective function definition:
    
    @objective(m, Max, sum((sum(P_PV[t,p,s] for s=ss)-N_scen*(1-eta_bat)*(P_bat_cha[t,p]+P_bat_dis[t,p])) for t=ts,p=ps));
    
    #----------------------------------------------------------------------------------------------------
    ### Solving Optimization:
    status = solve(m)

    #----------------------------------------------------------------------------------------------------
    ### Saving Data:
    
    # If Solved => Write Down the Results
    if string(status)!="Infeasible"
        write_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(interval)*".mat"; 
            status    = string(status),
            interval  = interval,
            P_bat_set = getvalue(P_bat_set[:,:]), 
            SOC_bat   = getvalue(SOC_bat[:,:]),
            P_PV_set  = getvalue(P_PV_set[:,:]),
            P_PV      = getvalue(P_PV[:,:,:]),
            ON_dies   = getvalue(ON_dies[:,:]),
            P_dies    = getvalue(P_dies[:,:,:,:]),
            P_bat_cha = getvalue(P_bat_cha[:,:]),
            P_bat_dis = getvalue(P_bat_dis[:,:]),
            X_bat     = getvalue(X_bat[:,:]),
            L_C       = L_C2)
    # If Infeasible -> Break!!!
    else 
        println("Loosening did not work -> Terminating Optimization!!!")
        break
    end
    
    # IJulia.clear_output(wait=false)
    println("Current progress: "*dec(my_int)*"/"*dec(N_EMS))
    
    my_int+=1
    
end

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-01, 1e+04]
Presolve removed 8647 rows and 5104 columns
Presolve time: 1.23s
Presolved: 187913 rows, 98288 columns, 655848 nonzeros
Variable types: 95696 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   32985    1.7352053e+07   1.182452e+06   0.000000e+00      5s
   61638    1.7318208e+07   7.177887e+05   0.000000e+00     10s
   86999    1.7273561e+07   5.225223e+05   0.000000e+00     15s
  110718    1.7158626e+07   3.862479e+05   0.000000e+00     20s
  135396    1.6887469e+07   2.242795e+05   0.000000e+00     25s
  156672    1.5628917e+07   8.858852e+05   0.000000e+00     30s
  176307  

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 108.575836343

Presolve removed 8654 rows and 5105 columns
Presolve time: 1.38s
Presolved: 187906 rows, 98287 columns, 655840 nonzeros
Variable types: 95695 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   26647    1.7326178e+07   1.232744e+06   0.000000e+00      5s
   56305    1.7298553e+07   8.167664e+05   0.000000e+00     10s
   84164    1.7255086e+07   5.562725e+05   0.000000e+00     15s
  110384    1.7164578e+07   4.343745e+05   0.000000e+00     20s
  133632    1.6944711e+07   2.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 571.454010636

Presolve removed 8667 rows and 5114 columns
Presolve time: 1.45s
Presolved: 187893 rows, 98278 columns, 655818 nonzeros
Variable types: 95686 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   23371    1.7220049e+07   1.304112e+06   0.000000e+00      5s
   52557    1.7196958e+07   7.652438e+05   0.000000e+00     10s
   80153    1.7164329e+07   5.756416e+05   0.000000e+00     15s
  103219    1.7105796e+07   4.144731e+05   0.000000e+00     20s
  126325    1.6930426e+07   2.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 98.994084908

Presolve removed 8682 rows and 5139 columns
Presolve time: 1.42s
Presolved: 187878 rows, 98253 columns, 655778 nonzeros
Variable types: 95661 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   26683    1.7487839e+07   1.176745e+06   0.000000e+00      5s
   54908    1.7465940e+07   7.865371e+05   0.000000e+00     10s
   81508    1.7436168e+07   5.574049e+05   0.000000e+00     15s
  105966    1.7352446e+07   4.378238e+05   0.000000e+00     20s
  129156    1.7159222e+07   3.0

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 555.250536878

Presolve removed 8690 rows and 5141 columns
Presolve time: 1.46s
Presolved: 187870 rows, 98251 columns, 655768 nonzeros
Variable types: 95659 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   25836    1.7282597e+07   1.215623e+06   0.000000e+00      5s
   54880    1.7259085e+07   8.129345e+05   0.000000e+00     10s
   80332    1.7222577e+07   5.885290e+05   0.000000e+00     15s
  103494    1.7162321e+07   3.853663e+05   0.000000e+00     20s
  128130    1.6947831e+07   2.

...

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 412.924010778

Presolve removed 8689 rows and 5151 columns
Presolve time: 1.42s
Presolved: 187871 rows, 98241 columns, 655759 nonzeros
Variable types: 95649 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7458640e+07   7.943125e+06   0.000000e+00      5s
   47211    1.7292261e+07   8.696437e+05   0.000000e+00     10s
   75286    1.7255586e+07   6.034227e+05   0.000000e+00     15s
   98321    1.7198503e+07   4.134089e+05   0.000000e+00     20s
  123004    1.7007280e+07   2.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [2e-01, 2e+04]
  RHS range        [2e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 1007.900476876

Presolve removed 8687 rows and 5139 columns
Presolve time: 1.45s
Presolved: 187873 rows, 98253 columns, 655773 nonzeros
Variable types: 95661 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7594336e+07   7.841513e+06   0.000000e+00      6s
   43911    1.7436094e+07   9.111564e+05   0.000000e+00     10s
   73440    1.7396709e+07   6.406005e+05   0.000000e+00     15s
   97153    1.7338032e+07   4.495237e+05   0.000000e+00     20s
  122050    1.7149003e+07   3

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 312.766692128

Presolve removed 8684 rows and 5132 columns
Presolve time: 1.41s
Presolved: 187876 rows, 98260 columns, 655783 nonzeros
Variable types: 95668 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7573421e+07   7.848099e+06   0.000000e+00      5s
   46011    1.7421028e+07   8.875532e+05   0.000000e+00     10s
   72095    1.7389135e+07   6.069477e+05   0.000000e+00     15s
   95249    1.7344773e+07   4.821389e+05   0.000000e+00     20s
  120455    1.7160212e+07   2.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 71.649531655

Presolve removed 8680 rows and 5133 columns
Presolve time: 1.49s
Presolved: 187880 rows, 98259 columns, 655786 nonzeros
Variable types: 95667 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7556106e+07   7.770140e+06   0.000000e+00      5s
   46901    1.7386288e+07   8.829403e+05   0.000000e+00     10s
   74987    1.7344471e+07   6.556716e+05   0.000000e+00     15s
   95400    1.7303182e+07   4.533980e+05   0.000000e+00     20s
  120427    1.7132722e+07   3.3

Excessive output truncated after 526250 bytes.